In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline

# Load data

In [2]:
path = "Data/"
sample_submission = pd.read_csv(path + "sample_submission.csv")
app_events = pd.read_csv(path + "app_events.csv")
app_labels = pd.read_csv(path + "app_labels.csv")
events = pd.read_csv(path + "events.csv")
gender_age_test = pd.read_csv(path + "gender_age_test.csv")
gender_age_train = pd.read_csv(path + "gender_age_train.csv")
phone_brand_device_model = pd.read_csv(path + "phone_brand_device_model.csv")

# Feature engineering

In [13]:
sample_submission.head(5)

,device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+
0,1002079943728939269,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833
1,-1547860181818787117,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833
2,7374582448058474277,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833
3,-6220210354783429585,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833
4,-5893464122623104785,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833
